# 10.11 Homework

In this homework, we'll deploy the churn preduction model from session 5.
We already have a docker image for this model - we'll use it for 
deploying the model to Kubernetes.


## Bulding the image

Clone the course repo if you haven't:



In [1]:
!git clone https://github.com/alexeygrigorev/mlbookcamp-code.git


Cloning into 'mlbookcamp-code'...
remote: Enumerating objects: 2222, done.
remote: Counting objects: 100% (1228/1228), done.
remote: Compressing objects: 100% (465/465), done.
remote: Total 2222 (delta 873), reused 987 (delta 733), pack-reused 994
Receiving objects: 100% (2222/2222), 7.20 MiB | 6.47 MiB/s, done.
Resolving deltas: 100% (1326/1326), done.



Go to the `course-zoomcamp/05-deployment/code` folder and 
execute the following:


```bash
docker build -t churn-model:v001 .
```




> **Note:** If you have troubles building the image, you can 
> use the image I built and published to docker hub:
> `agrigorev/zoomcamp-model:churn-v001`

Run it to test that it's working locally:

```bash
docker run -it --rm -p 9696:9696 churn-model:v001
```

And in another terminal, execute `predict-test.py` file:

```bash
python predict-test.py
```

Running predict-test.py:

In [14]:
! python "/home/elegant00/Dropbox/Ml_Zoomcamp/ML/ML-Zoomcamp-Homeworks/WEEK 10/mlbookcamp-code/course-zoomcamp/05-deployment/code/predict-test.py"


{'churn': False, 'churn_probability': 0.3257561103397851}
not sending promo email to xyz-123


Installing kubectl and kind



## Quesion 1: Version of kind

What's the version of `kind` that you have? 

Use `kind --version` to find out.

In [7]:
!kind --version

kind version 0.11.1


# Question 2


In [8]:
!kubectl get service


NAME         TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   2m5s


# Question 3: Uploading the image to kind


To be able to use the docker image we previously created (churn-model:v001), we need to register it with kind.

What's the command we need to run for that?



In [9]:
!kind load docker-image churn-model:v001

Image: "churn-model:v001" with ID "sha256:9cde0c4d564d44713cec545679d0cef53d780b755269e731b88ed16132eeaa7b" not yet present on node "kind-control-plane", loading...




## Question 4: Creating a deployment

Now let's create a deployment (e.g. `deployment.yaml`):

```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: churn
spec:
  selector:
    matchLabels:
      app: churn
  template:
    metadata:
      labels:
        app: churn
    spec:
      containers:
      - name: churn
        image: <Image>
        resources:
          limits:
            memory: "128Mi"
            cpu: "500m"
        ports:
        - containerPort: <Port>
```

Replace `<Image>` and `<Port>` with the correct values.

What is the value for `<Port>`?

Answer: It should be 9696

# Question 5: Pod name
Apply this deployment:



In [17]:
!!kubectl get pod 

['NAME                     READY   STATUS    RESTARTS   AGE',
 'churn-8449c67c88-2625s   1/1     Running   0          30s']

<b> Answer: </b> churn-8449c67c88-2625s


## Question 6: Creating a service 

Let's create a service for this deployment (`service.yaml`):

```yaml
apiVersion: v1
kind: Service
metadata:
  name: <Service name>
spec:
  type: LoadBalancer
  selector:
    app: <???>
  ports:
  - port: 80
    targetPort: <PORT>
```

Fill it in. What do we need to write instead of `<???>`?


<b> Answer: </b> churn

In [22]:
!kubectl port-forward service/churn-service 9696:80


Forwarding from 127.0.0.1:9696 -> 9696
Forwarding from [::1]:9696 -> 9696
Handling connection for 9696
^C


In [23]:
!python "/home/elegant00/Dropbox/Ml_Zoomcamp/ML/ML-Zoomcamp-Homeworks/WEEK 10/mlbookcamp-code/course-zoomcamp/05-deployment/code/predict-test.py"



{'churn': False, 'churn_probability': 0.3257561103403336}
not sending promo email to xyz-123
